In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/NLP_projetcs/research'

In [3]:
### change the dir to the root directory

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/bhikipallai/Desktop/Projects/NLP_projetcs'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# this peace of code is for the entity where we just store the data paths only, once configuration manager will execute then it will return the and store the data as per the entity data storage path

In [29]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml_file, create_directories

In [34]:
# this peace of code is for the configuration manager where we will read the config/params yaml and and return the data to the entity folder, this will used in pipeline 
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        # these 2 lines are for to read the path of config.yaml and params.yaml

        create_directories([self.config.artifacts_root])
        # this line of code is for to create the artifacts folder using the create_directories function from the common.py and using the DataIngestionConfig variables

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [35]:
import os
import zipfile
import urllib.request as request
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [36]:
# this peace of code is for the zip unzip the data and store the data into the respective folder, once the configuration manager will execute then it will return the and store the data as per the entity data storage path
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [37]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-20 09:10:16,637: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-20 09:10:16,640: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-20 09:10:16,641: INFO: common: created directory at: artifacts]
[2024-08-20 09:10:16,641: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-20 09:10:19,683: INFO: 300150182: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6B57:97B1A:107A80:163965:66C41021
Accept-Ranges: bytes
Date: Tue, 20 Aug 2024 03:40:19 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10250-MAA
X-Cache: MIS